In [60]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [61]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [62]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [63]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [64]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [65]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2015-07-01 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


4712
169


In [66]:
print(training_set)

      problem_id  user_id status       submissionDate
0             10        5     AC  2014-02-17 15:27:07
1              2        6     AC  2014-02-17 15:39:17
2              2        9     AC  2014-02-18 00:30:14
3             10        9     AC  2014-02-18 00:34:46
4              4        9     AC  2014-02-18 00:50:28
5              6        9     AC  2014-02-18 00:52:11
6             13        9     AC  2014-02-18 00:53:40
9             15        8     AC  2014-02-19 19:58:03
10             4        8     AC  2014-02-20 14:23:30
11            39       16     AC  2014-02-20 15:44:33
12            39       17     AC  2014-02-20 16:53:34
13            13       12     AC  2014-02-21 11:08:38
15            33       12     AC  2014-02-21 11:58:32
16            39       12     AC  2014-02-21 12:05:49
17            44       12     AC  2014-02-21 12:52:44
18            44       15     AC  2014-02-21 14:51:52
19            44       22     AC  2014-02-21 14:58:55
21            19        8   

In [67]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2015-07-01 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
6933          117      725     WA  2015-07-01 09:17:38
6934          122      725    RTE  2015-07-01 11:09:50
6935            2     1665     CE  2015-07-01 13:15:06
6936          340     1504     WA  2015-07-01 16:12:24
6937          443     1735     TL  2015-07-01 18:55:30
6938           39     1737     AC  2015-07-01 20:36:18
6939          134     1317     AC  2015-07-02 13:37:18
6940          150     1504     AC  2015-07-02 19:43:11
6941          183     1504     AC  2015-07-02 20:37:12
6942           39     1736     AC  2015-07-03 00:00:53
6943          195     1504     AC  2015-07-03 03:53:16
6944          508     1504     AC  2015-07-03 04:19:22
6945            2     1504     AC  2015-07-03 04:29:37
6946          174     1504     AC  2015-07-03 17:16:06
6947           35      103     AC  2015-07-03 18:10:52
6948          181     1504     AC  2015-07-03 22:08:51
6949            8     1736     AC  2015-07-04 15:46:08
6950      

In [68]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

4443


In [69]:
def apply_wcn(row, graph):
    """
        Funcion que devuelve para cada par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """
    
    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de wcn para el par de nodos
    value = nx.adamic_adar_index(graph, [(row['one'], row['two'])])
    
    value_wcn = 0
    for u, v, p in value:
        # itero el iterador, guardando el valor de adar adamic
        value_wcn = p
    
    return value_wcn

def create_wcn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de wcn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_wcn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_wcn['wcn'] = dataFrame_wcn.apply (lambda row: apply_wcn(row, graph), axis=1)


    return dataFrame_wcn

In [70]:
wcn_df = create_wcn_data(graph, nodes)
print(wcn_df)

       one  two        wcn
0       10    2  11.304202
1       10    4   9.811833
2       10    6   8.521302
3       10   13   9.431564
4       10   15  10.654518
5       10   39  11.311524
6       10   33   9.944589
7       10   44  10.162211
8       10   19   6.680407
9       10   27   5.449414
10      10   60   5.103790
11      10  100  10.519732
12      10   93   0.000000
13      10   53   9.234789
14      10   51   9.199781
15      10   49  10.850358
16      10   70   9.919868
17      10   62  10.955312
18      10   35   8.038641
19      10   81   1.066350
20      10    8   7.491605
21      10   17   1.747898
22      10   23   5.102796
23      10   25   0.000000
24      10   86   3.308712
25      10   29  10.167451
26      10   31   0.000000
27      10   47   0.000000
28      10   65   0.000000
29      10   73   6.412311
...    ...  ...        ...
14166  510  509   6.895348
14167  510  507   0.000000
14168  511  504   0.000000
14169  511  503   7.266353
14170  511  505   2.631301
1

In [71]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                       list_problem_id  user_id
0                                        [10, 76, 489]        5
1                                                  [2]        6
2                     [39, 44, 76, 105, 109, 114, 117]        7
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
5                                            [33, 109]       11
6    [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
7                                            [44, 187]       15
8                                                 [39]       16
9                                             [39, 86]       17
10                                                [44]       22
11                                           [100, 13]       24
12                 [258, 141, 309, 310, 390, 187, 510]       25
13                                               [100]       28
14                                      

In [72]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [73]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             3               2
1           6             1               1
2           9             9               5
3           8            40               8
4          16             1               0
5          17             2               2
6          12            12               0
7          18             0              13
8          15             2               2
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            40               2
16         33             7               2
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [74]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
2           9             9               5
3           8            40               8
29         60             6              27
118        25             7              33
261       130            62              34
274       382             5               6
285       206           111              38
286        62           112              32
299       414             9             103
310       437           134              13
316       443            41              22
317       448            31              33
333       381            13              10
348       103           109              48
383       542             8              13
395       431            11               8
483       689             5              41
597       254            11              12
601       864            14               8
619       912             7              17
621       840            26              19
633       935            88     

In [75]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                       list_problem_id  user_id
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
12                 [258, 141, 309, 310, 390, 187, 510]       25
37                            [100, 62, 2, 57, 53, 23]       60
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381
225                          [119, 262, 264, 114, 388]      382
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      431
248  [250, 124, 249, 100, 4, 122, 57, 30

In [76]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [77]:
def getSimilarProblems(row, wcn_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor wcn
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = wcn_df[wcn_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['wcn'] > 0]
    column_result_two_tmp = wcn_df[wcn_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['wcn'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_wcn = list_one + list_two
    
    # print(list_wcn)
    
    if list_wcn == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_wcn = graph.nodes()
        list_wcn.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_wcn

In [78]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, wcn_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...
13           93  [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10

In [79]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17                              [35, 133, 129, 437, 103]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                             [35, 103]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [80]:
def wcn_value(one, two, graph):
    """
        Funcion que devuelve para ese par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """

    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de wcn para el par de nodos
    value = nx.adamic_adar_index(graph, [(one, two)])
    # print(value)
    value_wcn = -1
    for u, v, p in value:
        # print(p)
        # itero el iterador, guardando el valor de adar adamic
        value_wcn = p

    return value_wcn

In [81]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los valores de wcn de todos los nodos con los que tiene wcn
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    problem = row['problem_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + wcn_value(problem, elem, graph)
        
    return suma

In [82]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada problema vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con problem_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    problem = row['problem_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = wcn_value(problem, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [83]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_problem_recomend['total_weight'] = dataframe_problem_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [84]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

dataframe_problem_recomend['score'] = dataframe_problem_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [85]:

# elimino la columna de total_weight, ya que no interesa
del dataframe_problem_recomend['total_weight']

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [86]:
def getListUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de usuarios que han
        hecho los problemas similares a ese. Además eliminara los usuarios que ya hayan hecho el problema
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese problema
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de problemas vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_users_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_users_df['list_user_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [87]:
dataframe_problem_recomend['lista_users_por_problema'] = dataframe_problem_recomend.apply(lambda row: getListUsersFromSimilarProblems(row, df_users_recommend), axis=1)

print(dataframe_problem_recomend)


     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 53, 51, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 53, 51, 49, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 53, 51, 49, 70, ...   
6            39  [33, 44, 19, 27, 60, 100, 53, 51, 49, 70, 62, ...   
7            33  [44, 19, 27, 60, 100, 53, 51, 49, 70, 62, 35, ...   
8            44  [19, 27, 60, 100, 53, 51, 49, 70, 62, 35, 81, ...   
9            19  [27, 60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
10           27  [60, 100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
11           60  [100, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 8...   
12          100  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 29...   
13           93  [25

In [88]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation_neigh = dataframe_problem_recomend.groupby(['problem_id']).neighbors.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation_neigh.columns = ['problem_id', 'neighbors', ]

print(df_separation_neigh)

       problem_id  neighbors
0               2          4
1               2          6
2               2         13
3               2         15
4               2         39
5               2         33
6               2         44
7               2         19
8               2         27
9               2         60
10              2        100
11              2         53
12              2         51
13              2         49
14              2         70
15              2         62
16              2         35
17              2         81
18              2          8
19              2         17
20              2         23
21              2         86
22              2         29
23              2         73
24              2         95
25              2         78
26              2        105
27              2         97
28              2         57
29              2         83
...           ...        ...
23454         511        336
23455         511        342
23456         

In [89]:
# creo un nuevo dataframe que agrupa por el usuario, poniendo en la columna de al lado, el score
df_separation_pond = dataframe_problem_recomend.groupby(['problem_id']).score.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation_pond.columns = ['problem_id', 'score']

print(df_separation_pond)


       problem_id     score
0               2  0.010884
1               2  0.007955
2               2  0.011002
3               2  0.012265
4               2  0.013653
5               2  0.011774
6               2  0.012366
7               2  0.005359
8               2  0.004323
9               2  0.003310
10              2  0.012214
11              2  0.010839
12              2  0.010082
13              2  0.013091
14              2  0.011613
15              2  0.013010
16              2  0.008414
17              2  0.000825
18              2  0.005769
19              2  0.000987
20              2  0.004130
21              2  0.001966
22              2  0.011740
23              2  0.005492
24              2  0.001064
25              2  0.009111
26              2  0.011628
27              2  0.001210
28              2  0.005621
29              2  0.008864
...           ...       ...
23454         511  0.008124
23455         511  0.007234
23456         511  0.009239
23457         511  0

In [90]:
df_separation = df_separation_neigh

df_separation['score'] = df_separation_pond['score']

# creo ahora el nuevo dataframe gracias a los dos df anteriores que eran auxiliares
print(df_separation)


       problem_id  neighbors     score
0               2          4  0.010884
1               2          6  0.007955
2               2         13  0.011002
3               2         15  0.012265
4               2         39  0.013653
5               2         33  0.011774
6               2         44  0.012366
7               2         19  0.005359
8               2         27  0.004323
9               2         60  0.003310
10              2        100  0.012214
11              2         53  0.010839
12              2         51  0.010082
13              2         49  0.013091
14              2         70  0.011613
15              2         62  0.013010
16              2         35  0.008414
17              2         81  0.000825
18              2          8  0.005769
19              2         17  0.000987
20              2         23  0.004130
21              2         86  0.001966
22              2         29  0.011740
23              2         73  0.005492
24              2        

In [91]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada problema
# ahora lo que quiero es ordenar los vecinos por cada problema en funcion de su ponderacion
# primero ordeno por su valor de problem y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['problem_id', 'score'], ascending=False)
print(df_separation)

       problem_id  neighbors     score
23410         511        254  0.012123
23442         511        307  0.012120
23444         511        309  0.012118
23462         511        388  0.012117
23463         511        390  0.012113
23477         511        471  0.012107
23467         511        441  0.012103
23448         511        315  0.011701
23446         511        314  0.011322
23357         511         39  0.011290
23392         511        134  0.011286
23368         511         62  0.011230
23468         511        436  0.011228
23363         511        100  0.011227
23436         511        233  0.011222
23408         511        259  0.011218
23458         511        352  0.011212
23472         511        443  0.011204
23385         511        109  0.010888
23411         511        258  0.010881
23447         511        316  0.010809
23431         511        191  0.010808
23469         511        444  0.010785
23445         511        310  0.010683
23476         511        

In [92]:
# elimino la columna ya que no interesa
del df_separation['score']

print(df_separation)

       problem_id  neighbors
23410         511        254
23442         511        307
23444         511        309
23462         511        388
23463         511        390
23477         511        471
23467         511        441
23448         511        315
23446         511        314
23357         511         39
23392         511        134
23368         511         62
23468         511        436
23363         511        100
23436         511        233
23408         511        259
23458         511        352
23472         511        443
23385         511        109
23411         511        258
23447         511        316
23431         511        191
23469         511        444
23445         511        310
23476         511        465
23422         511        226
23409         511        256
23367         511         70
23412         511        241
23418         511        203
...           ...        ...
131             2        511
48              2        130
75            

In [93]:
# ahora tengo que hacer un nuevo dataframe con problem_id y vecino
# hago primero la agrupacion por problem_id
grouped_r = df_separation.groupby('problem_id')

# hago la agregacion en una lista 
df_aux = grouped_r.aggregate(lambda x:list(x))

print(df_aux)

                                                    neighbors
problem_id                                                   
2           [109, 39, 258, 254, 49, 62, 259, 256, 134, 44,...
4           [109, 39, 49, 62, 2, 259, 258, 254, 100, 15, 2...
6           [109, 39, 49, 2, 259, 258, 44, 62, 33, 134, 10...
8           [109, 39, 62, 2, 258, 254, 256, 134, 49, 33, 7...
10          [109, 39, 2, 62, 254, 49, 15, 259, 258, 256, 1...
13          [39, 109, 49, 62, 2, 44, 203, 134, 33, 258, 25...
15          [109, 39, 62, 49, 258, 254, 2, 259, 134, 256, ...
17          [105, 57, 83, 134, 259, 256, 254, 241, 191, 23...
19          [254, 109, 39, 62, 49, 15, 100, 2, 259, 258, 3...
23          [109, 105, 39, 62, 49, 203, 134, 256, 258, 316...
25          [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...
27          [109, 39, 258, 100, 49, 2, 4, 256, 70, 134, 25...
29          [39, 109, 62, 100, 2, 49, 134, 203, 15, 44, 25...
31          [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...
33      

In [94]:
#print(sorted(nodes))

# voy a crear un nuevo dataframe con la columna problem_id y neighbors (con el anterior no se puede trabajar sin indices)
df_recommend = pd.DataFrame({'problem_id':sorted(nodes), 'neighbors':df_aux['neighbors'].tolist()})

print(df_recommend)

                                             neighbors  problem_id
0    [109, 39, 258, 254, 49, 62, 259, 256, 134, 44,...           2
1    [109, 39, 49, 62, 2, 259, 258, 254, 100, 15, 2...           4
2    [109, 39, 49, 2, 259, 258, 44, 62, 33, 134, 10...           6
3    [109, 39, 62, 2, 258, 254, 256, 134, 49, 33, 7...           8
4    [109, 39, 2, 62, 254, 49, 15, 259, 258, 256, 1...          10
5    [39, 109, 49, 62, 2, 44, 203, 134, 33, 258, 25...          13
6    [109, 39, 62, 49, 258, 254, 2, 259, 134, 256, ...          15
7    [105, 57, 83, 134, 259, 256, 254, 241, 191, 23...          17
8    [254, 109, 39, 62, 49, 15, 100, 2, 259, 258, 3...          19
9    [109, 105, 39, 62, 49, 203, 134, 256, 258, 316...          23
10   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...          25
11   [109, 39, 258, 100, 49, 2, 4, 256, 70, 134, 25...          27
12   [39, 109, 62, 100, 2, 49, 134, 203, 15, 44, 25...          29
13   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...        

In [95]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los usuarios que ya ha hechon el problema
        list_users = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_users
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [96]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
df_recommend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(df_recommend)

                                             neighbors  problem_id  \
0    [109, 39, 258, 254, 49, 62, 259, 256, 134, 44,...           2   
1    [109, 39, 49, 62, 2, 259, 258, 254, 100, 15, 2...           4   
2    [109, 39, 49, 2, 259, 258, 44, 62, 33, 134, 10...           6   
3    [109, 39, 62, 2, 258, 254, 256, 134, 49, 33, 7...           8   
4    [109, 39, 2, 62, 254, 49, 15, 259, 258, 256, 1...          10   
5    [39, 109, 49, 62, 2, 44, 203, 134, 33, 258, 25...          13   
6    [109, 39, 62, 49, 258, 254, 2, 259, 134, 256, ...          15   
7    [105, 57, 83, 134, 259, 256, 254, 241, 191, 23...          17   
8    [254, 109, 39, 62, 49, 15, 100, 2, 259, 258, 3...          19   
9    [109, 105, 39, 62, 49, 203, 134, 256, 258, 316...          23   
10   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...          25   
11   [109, 39, 258, 100, 49, 2, 4, 256, 70, 134, 25...          27   
12   [39, 109, 62, 100, 2, 49, 134, 203, 15, 44, 25...          29   
13   [256, 257, 2, 2

In [97]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [98]:
# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
df_recommend['lista_users_unique'] = df_recommend.apply(lambda row: delRepetitions(row), axis=1)

print(df_recommend)

                                             neighbors  problem_id  \
0    [109, 39, 258, 254, 49, 62, 259, 256, 134, 44,...           2   
1    [109, 39, 49, 62, 2, 259, 258, 254, 100, 15, 2...           4   
2    [109, 39, 49, 2, 259, 258, 44, 62, 33, 134, 10...           6   
3    [109, 39, 62, 2, 258, 254, 256, 134, 49, 33, 7...           8   
4    [109, 39, 2, 62, 254, 49, 15, 259, 258, 256, 1...          10   
5    [39, 109, 49, 62, 2, 44, 203, 134, 33, 258, 25...          13   
6    [109, 39, 62, 49, 258, 254, 2, 259, 134, 256, ...          15   
7    [105, 57, 83, 134, 259, 256, 254, 241, 191, 23...          17   
8    [254, 109, 39, 62, 49, 15, 100, 2, 259, 258, 3...          19   
9    [109, 105, 39, 62, 49, 203, 134, 256, 258, 316...          23   
10   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...          25   
11   [109, 39, 258, 100, 49, 2, 4, 256, 70, 134, 25...          27   
12   [39, 109, 62, 100, 2, 49, 134, 203, 15, 44, 25...          29   
13   [256, 257, 2, 2

In [99]:
# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
new_df_separation = df_recommend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

new_df_separation.columns = ['problem_id', 'possible_user']

print(new_df_separation)

        problem_id  possible_user
0                2              6
1                2             33
2                2             67
3                2             56
4                2             84
5                2             60
6                2            112
7                2            124
8                2            106
9                2             41
10               2            158
11               2            114
12               2            167
13               2            165
14               2            149
15               2            217
16               2            218
17               2            191
18               2             44
19               2            205
20               2            245
21               2            246
22               2            251
23               2            189
24               2            216
25               2            135
26               2            294
27               2            177
28            

In [100]:
def getScoring(row, dataframe_problem_recomend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de usuarios que han hecho los vecinos del problema
    list_problems_users_df = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    list_user_per_problem = list(list_problems_users_df['lista_users_por_problema'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_problem = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el usuario del que quiero calcular el scoring
    user = row['possible_user']
    # print(problem)
    
    suma = 0
    k = len(list_user_per_problem)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_user_per_problem[i]
        # print(lista_a_comprobar)
        if user in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + (1/(i+1))
    
    return suma

In [101]:
# ahora voy a calcular el scoring para cada problema-usuario
new_df_separation['score'] = new_df_separation.apply(lambda row: getScoring(row, dataframe_problem_recomend), axis=1)

print(new_df_separation)

        problem_id  possible_user     score
0                2              6  0.000000
1                2             33  0.000000
2                2             67  0.000000
3                2             56  0.000000
4                2             84  0.000000
5                2             60  0.000000
6                2            112  0.000000
7                2            124  0.000000
8                2            106  0.000000
9                2             41  0.000000
10               2            158  0.000000
11               2            114  0.000000
12               2            167  0.000000
13               2            165  0.000000
14               2            149  0.000000
15               2            217  0.000000
16               2            218  0.000000
17               2            191  0.000000
18               2             44  0.000000
19               2            205  0.000000
20               2            245  0.000000
21               2            24

In [102]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
new_df_separation = new_df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(new_df_separation)

        problem_id  possible_user     score
18638           53           1733  1.000000
33849          100           1733  0.500000
20430           60           1733  0.333333
9121            27           1733  0.250000
6548            19           1733  0.200000
15133           44           1733  0.166667
11738           33           1733  0.142857
14277           39           1733  0.125000
5223            15           1733  0.111111
4646            13           1733  0.100000
2120             6           1733  0.090909
1286             4           1733  0.083333
518              2           1733  0.076923
3795            10           1733  0.071429
145262         509           1733  0.066667
138947         502           1733  0.062500
141740         505           1733  0.058824
139842         503           1733  0.055556
130899         469           1733  0.052632
147159         511           1733  0.052632
146161         510           1733  0.050000
144451         508           173

In [103]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = new_df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                            problem_id  user_id
0    [509, 502, 505, 503, 511, 510, 508, 487, 25, 3...        5
1    [10, 509, 25, 31, 47, 65, 76, 93, 166, 231, 23...        6
2    [322, 78, 33, 13, 95, 83, 6, 73, 57, 4, 29, 97...        7
3    [254, 255, 187, 509, 130, 31, 47, 266, 331, 26...        8
4    [509, 502, 505, 70, 191, 97, 503, 25, 31, 266,...        9
5    [39, 88, 15, 90, 322, 13, 83, 6, 57, 4, 97, 2,...       11
6    [60, 6, 27, 4, 171, 2, 509, 150, 502, 505, 503...       12
7    [33, 147, 39, 130, 15, 171, 13, 150, 6, 159, 4...       15
8    [13, 6, 4, 2, 10, 509, 502, 282, 505, 503, 511...       16
9    [23, 13, 17, 6, 8, 4, 81, 2, 10, 35, 509, 62, ...       17
10   [33, 39, 15, 13, 6, 4, 2, 10, 509, 502, 505, 5...       22
11   [6, 60, 4, 27, 2, 19, 509, 44, 502, 505, 33, 5...       24
12   [308, 147, 254, 388, 508, 136, 130, 279, 374, ...       25
13   [60, 27, 19, 44, 33, 39, 15, 13, 6, 4, 2, 10, ...       28
14   [60, 27, 19, 44, 33, 39, 15, 13, 6,

In [104]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [105]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                       list_problem_id  user_id  \
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8   
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9   
12                 [258, 141, 309, 310, 390, 187, 510]       25   
37                            [100, 62, 2, 57, 53, 23]       60   
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62   
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103   
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105   
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206   
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254   
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381   
225                          [119, 262, 264, 114, 388]      382   
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414   
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      43

In [106]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

     user_id                                     recommendation
3          8  [254, 255, 187, 509, 130, 31, 47, 266, 331, 26...
4          9  [509, 502, 505, 70, 191, 97, 503, 25, 31, 266,...
12        25  [308, 147, 254, 388, 508, 136, 130, 279, 374, ...
37        60  [10, 70, 17, 27, 97, 19, 509, 49, 44, 8, 502, ...
38        62  [122, 365, 469, 311, 446, 264, 57, 95, 279, 2,...
69       103  [31, 266, 484, 93, 379, 166, 488, 239, 489, 22...
71       105  [404, 444, 397, 88, 226, 264, 509, 393, 390, 9...
88       130  [122, 195, 509, 226, 374, 316, 308, 213, 23, 1...
135      206  [469, 365, 122, 446, 264, 279, 25, 31, 65, 76,...
162      254  [253, 257, 256, 162, 241, 49, 251, 243, 206, 3...
224      381  [203, 257, 254, 88, 122, 53, 237, 217, 314, 19...
225      382  [181, 122, 374, 183, 109, 124, 365, 88, 224, 9...
237      414  [397, 444, 509, 88, 393, 209, 502, 390, 90, 38...
245      431  [51, 4, 206, 187, 257, 162, 254, 2, 53, 147, 1...
248      437  [484, 486, 44, 51, 441, 48

In [107]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [108]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

     user_id                                     recommendation  \
3          8  [254, 255, 187, 509, 130, 31, 47, 266, 331, 26...   
4          9  [509, 502, 505, 70, 191, 97, 503, 25, 31, 266,...   
12        25  [308, 147, 254, 388, 508, 136, 130, 279, 374, ...   
37        60  [10, 70, 17, 27, 97, 19, 509, 49, 44, 8, 502, ...   
38        62  [122, 365, 469, 311, 446, 264, 57, 95, 279, 2,...   
69       103  [31, 266, 484, 93, 379, 166, 488, 239, 489, 22...   
71       105  [404, 444, 397, 88, 226, 264, 509, 393, 390, 9...   
88       130  [122, 195, 509, 226, 374, 316, 308, 213, 23, 1...   
135      206  [469, 365, 122, 446, 264, 279, 25, 31, 65, 76,...   
162      254  [253, 257, 256, 162, 241, 49, 251, 243, 206, 3...   
224      381  [203, 257, 254, 88, 122, 53, 237, 217, 314, 19...   
225      382  [181, 122, 374, 183, 109, 124, 365, 88, 224, 9...   
237      414  [397, 444, 509, 88, 393, 209, 502, 390, 90, 38...   
245      431  [51, 4, 206, 187, 257, 162, 254, 2, 53, 147, 1..

In [109]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

     user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
3          8              [254]         [254, 255]    [254, 255, 187]   
4          9              [509]         [509, 502]    [509, 502, 505]   
12        25              [308]         [308, 147]    [308, 147, 254]   
37        60               [10]           [10, 70]       [10, 70, 17]   
38        62              [122]         [122, 365]    [122, 365, 469]   
69       103               [31]          [31, 266]     [31, 266, 484]   
71       105              [404]         [404, 444]    [404, 444, 397]   
88       130              [122]         [122, 195]    [122, 195, 509]   
135      206              [469]         [469, 365]    [469, 365, 122]   
162      254              [253]         [253, 257]    [253, 257, 256]   
224      381              [203]         [203, 257]    [203, 257, 254]   
225      382              [181]         [181, 122]    [181, 122, 374]   
237      414              [397]         [397, 444] 

In [110]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
2              [445, 436, 437, 441, 442, 443, 444, 62]        8
3                             [83, 109, 136, 141, 254]        9
9    [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...       25
16   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...       60
17   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...       62
21   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...      103
22   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...      105
24   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...      130
30   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...      206
34   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...      254
43    [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]      381
44                              [33, 8, 465, 6, 2, 13]      382
46   [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...      414
50             [253, 33, 252, 443, 444, 441, 404, 503]      431
51   [489, 181, 390, 393, 397, 437, 486,

In [111]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0             [445, 436, 437, 441, 442, 443, 444, 62]            [254]   
1                            [83, 109, 136, 141, 254]            [509]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...            [308]   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...             [10]   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...            [122]   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...             [31]   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...            [404]   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...            [122]   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...            [469]   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...            [253]   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]            [203]   
11                             [33, 8, 465, 6, 2, 13]            [181]   
12  [239, 62, 187, 206, 313, 316, 504,

In [112]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [113]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [114]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [115]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [116]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [117]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0             [445, 436, 437, 441, 442, 443, 444, 62]            [254]   
1                            [83, 109, 136, 141, 254]            [509]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...            [308]   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...             [10]   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...            [122]   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...             [31]   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...            [404]   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...            [122]   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...            [469]   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...            [253]   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]            [203]   
11                             [33, 8, 465, 6, 2, 13]            [181]   
12  [239, 62, 187, 206, 313, 316, 504,

In [118]:


f = open("C:/hlocal/TFM/vot_wcn", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.23809523809523808
Precision ----------
0.23809523809523808
Mrr  ----------
0.23809523809523808
Recall  ----------
0.009155375154762049
F1  ----------
0.01747562567281759
2
###########
One hit ----------
0.3333333333333333
Precision ----------
0.21428571428571427
Mrr  ----------
0.2857142857142857
Recall  ----------
0.014908599343329861
F1  ----------
0.027368064808243466
3
###########
One hit ----------
0.4523809523809524
Precision ----------
0.2460317460317461
Mrr  ----------
0.32539682539682546
Recall  ----------
0.02646958328383218
F1  ----------
0.04644139758828762
4
###########
One hit ----------
0.4523809523809524
Precision ----------
0.2261904761904762
Mrr  ----------
0.32539682539682546
Recall  ----------
0.032168518079875315
F1  ----------
0.0542779658969406
5
###########
One hit ----------
0.47619047619047616
Precision ----------
0.23809523809523805
Mrr  ----------
0.3301587301587302
Recall  ----------
0.04288330594604216
F1  ----------
0.06